In [96]:
import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import pandas as pd

# data loading
file_path = '/content/Quality_of_Life.csv'
data = pd.read_csv(file_path)

# Cleaning
data['Property Price to Income Value'] = pd.to_numeric(data['Property Price to Income Value'], errors='coerce')
data['Quality of Life Value'] = data['Quality of Life Value'].str.extract(r'([0-9.]+)').astype(float)
data.replace({'None': pd.NA, "'None'": pd.NA}, inplace=True)

# Dash app
app = dash.Dash(__name__)

# Layout
app.layout = html.Div([
    html.Div([
        html.H2("Quality of Life for Each Country", style={
            'textAlign': 'center',
            'color': '#FFFFFF',
            'marginBottom': '20px'
        }),
        dcc.Dropdown(
            id='num-dropdown',
            options=[{'label': col, 'value': col} for col in data.select_dtypes('float').columns],
            value='Cost of Living Value',
            placeholder="Select a numerical column...",
            style={
                'width': '80%',
                'margin': '0 auto',
                'padding': '10px',
                'borderRadius': '8px',
                'border': '1px solid #2c3e50',
                'backgroundColor': '#ecf0f1',
                'color': '#2c3e50',
                'fontSize': '16px'
            }
        )
    ], style={
        'display': 'flex',
        'flexDirection': 'column',
        'alignItems': 'center',
        'justifyContent': 'center',
        'padding': '20px'
    }),

    html.Div([
        dcc.Graph(id='num-graph', style={'width': '90%', 'margin': '10px auto'}),
        dcc.Graph(id='cat-graph', style={'width': '90%', 'margin': '10px auto'})
    ], style={
        'display': 'flex',
        'flexDirection': 'column',
        'alignItems': 'center'
    })
])

# Callbacks
@app.callback(
    [Output('num-graph', 'figure'), Output('cat-graph', 'figure')],
    [Input('num-dropdown', 'value')]
)
def update_graphs(num_column):
    # plot numerical data
    fig1 = px.line(data, x='country', y=num_column, title=f'Country vs {num_column}')

    # related categorical column
    cat_column = num_column.replace('Value', 'Category')
    if cat_column in data.columns:
        fig2 = px.bar(data, x='country', y=num_column, color=cat_column, title=f'Country vs {num_column} by {cat_column}')
    else:
        fig2 = px.bar(title="No related categorical data available")

    return fig1, fig2

# Run app
if __name__ == '__main__':
    app.run(debug=True)

<IPython.core.display.Javascript object>